<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/logo.png?raw=true" alt="2025年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

# 今回の勉強内容
　今回は、これまでの解析手法の紹介とは違って、もう少し具体的な操作を扱います。

バイオインフォマティクスのツールやプログラムは、WordやExcelの様にマウスでクリック等で操作するアプリと異なり、コマンドから操作をする場合が多いです。

コマンド自体を学習しようとすると扱いきれないので、今回は1例として配列データをコマンドやコマンドから使用するツールで扱う実習を行います。

---

# FASTAファイルを自由に使い回す方法について
　ゲノム配列は基本的に**FASTA**ファイルと呼ばれる形式で保存されています。

FASTA形式では、シンプルに2行で1つの配列データの情報を表します。

```
>配列の名前 (名前の後にスペースを空けて色々情報が書いてあることも)
TGAAGCAGT..............(リードの塩基配列)
```

　また、最初の方の講義でも扱いましたが、シーケンスを行って得られるリードのデータは**FASTQ**と呼ばれる４行で１つのリード情報を示す形式でファイルに保存されています。

```
@リードの名前
TGAAGCAGT..............(リードの塩基配列)
+
F1GCFCGGE..............(各塩基のクオリティ)
```

- 第1行（@からはじまる行）: リードの名前
- 第2行: リードの塩基配列
- 第3行（+からはじまる行）: 第2行と第4行の区切りのための行
- 第4行: 各塩基配列のクオリティ（ASCIIコードで書かれている）。1文字が何らかのクオリティスコア（数値）を示している。
 - 参考: https://bi.biopapyrus.jp/rnaseq/qc/fastq-quality-score.html


本日はこれらの配列データをコマンドなどから扱う方法を実習で行っていきます。

言ってしまえばFASTAファイルもFASTQファイルもテキストファイルなので、コピーペーストなど普通のテキストファイルの様に扱うことは出来ますが、コマンドや`seqkit`等のツールを使用することで、より効率的にFASTAファイルを扱うことが出来ます。





## コマンドについて

今回の実習では`コマンド`による操作を主に行います。

皆さんは普段マウスでクリック等を行うことでコンピュータを捜査している方が多いかと思いますが、`コマンド`と呼ばれるものでも同じようにコンピュータを操作することは可能です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/CUI_GUI.png?raw=true" alt="cui_gui" height="400px" align="middle">

そして、多くのバイオインフォマティクスのツールはコマンドから動かすことが多いです。

<br>

今使用しているGoogle Colaboratoryというサービスでも、PythonやR等で書かれたプログラムを実行するだけでは無く、

コマンドを使用してGoogle側のサーバー(コンピュータ)を操作することが出来ます。

<img src="https://lh3.googleusercontent.com/rGFAR5vGV9lYsnnb1QhFUZlWbOtHVTwhLq4wBjIIxgJ05Dldb6ELwBTpmkBM-1i7XQ1ir1JIkxK3fcfQ20HK0VkraOPPZdClaM9caTbvsewYMU-lvsIhvXvtcDuhm8VHL495z-m9Cq_oktsqtYg-EAZXpc6_-3ebHARqVPE-q2d6EM_fq6Zcnjq3BODbYdxmAyOrS0lycbTJBvcGNWAkAJ-ECJBPyKAevUEXSCUDx7NkYgjh1Wj02VTP2POVBmpJTZhegRVyWMVryUjeKeJd9mEcoCNRWV7dEk8O80ETmOh0512AuMqzLUivD6mGxItX24uAChpt64ToJq2lAvDeg2gYk7fIKPUGPvroLFKy_UN3dlKSeC1upQsdEwDHEcgp5RK3tggn1eERsLKxqY-QDjL-LlcRyYrRqSjiTWFIkNBeif3v0R8K80JN8R2b49ukgR-Y3jOJQ1Sq23Vt16Nz98dhAXGADKNXnnmPcoeLtoHkGXIO0XxNCXj568wyC4q4WiPt82ATSORKpM-EQis-tcjh840FMsJ0fuHe-jOtLYtWqbvnmXDYJBtX_K-HDu3Vf0tD6wWks2A10NTlyh_W9Id5KRmBfq5bpqxYhv21Knrl7EFYCyF3UofbJ5MZGvjuFu-4yUwW5m5d6KsgKzs2KIUSAvF6WtCH2Vqz-HynVTMx0Q1pyRDXw8QBThSW=w1341-h510-no?authuser=0" alt="colab_server" height="150px">

これまであまり細かくは触れてきませんでしたが、PythonやRのコードとは違って、

`!`で始まっていたり、`％％bash`と書かれていた所はコマンドを動かしている形になります。

試しに`screenfetch`というコマンドで動かすツールを動かしてみます。

In [ ]:
# screenfetchという操作しているコンピュータのスペックを表示するツールをインストールするコマンド
!apt-get install screenfetch

In [ ]:
# screenfetchを使用
!screenfetch

In [ ]:
# %%bashをつけてもコマンドは動かせる(複数行の場合はこちらが楽)
%%bash
screenfetch

# 今回の実習内容

今回はいくつかのコマンド、`seqkit`と`blast`というツールを使って以下の様な実習を行います。

1. 必要な情報をFASTAから抜き出す(seqkit)
1. 遺伝子Aと相同性のある配列をFASTAから抜き出す(blast, seqkit)
1. 指定領域の配列をFASTAから抜き出す(seqkit)
1. 遺伝子のアノテーション情報(gffファイル)から遺伝子の領域を抜き出す(gffread)
1. seqkitの他の機能





### データのダウンロード
　この実習で使用するサンプルデータをダウンロードします。ここでは公開されているゲノム配列を使用します。

RNAseqによる遺伝子アノテーションの所でも少しお話ししましたが、既にシーケンスによって決められたゲノム配列や遺伝子のアノテーション情報、各遺伝子のアミノ酸配列等は、[Ensemble](https://asia.ensembl.org/info/data/ftp/index.html)や[Ensemble plants](https://plants.ensembl.org/info/data/ftp/index.html), [NCBI](https://ncbi.nlm.nih.gov/datasets/genome/)等のデータベースに登録されて公開されています。

[Ensemble](https://asia.ensembl.org/info/data/ftp/index.html)や[Ensemble plants](https://plants.ensembl.org/info/data/ftp/index.html), [NCBI](https://ncbi.nlm.nih.gov/datasets/genome/)のサーバーにあるデータは、`wget`というサーバーからファイルを取得するコマンドでダウンロードできます。
```
wget ダウンロードリンク
```

例として[イネいもち病菌のゲノム配列](https://ftp.ensemblgenomes.ebi.ac.uk/pub/fungi/release-60/fasta/magnaporthe_oryzae/)をダウンロードするコマンドを下に表示していますが、

同様のアプローチで自分の興味のある生物の配列をダウンロードすることが可能です。

In [ ]:
# ダウンロードするコマンド
# DNA配列のダウンロード
!wget https://ftp.ensemblgenomes.ebi.ac.uk/pub/fungi/release-60/fasta/magnaporthe_oryzae/dna/Magnaporthe_oryzae.MG8.dna.toplevel.fa.gz
# 各遺伝子のアミノ酸配列のダウンロード
!wget https://ftp.ensemblgenomes.ebi.ac.uk/pub/fungi/release-60/fasta/magnaporthe_oryzae/pep/Magnaporthe_oryzae.MG8.pep.all.fa.gz
# 遺伝子のアノテーション情報のダウンロード
!wget https://ftp.ensemblgenomes.ebi.ac.uk/pub/fungi/release-60/gtf/magnaporthe_oryzae/Magnaporthe_oryzae.MG8.60.gtf.gz

配列データ(FASTAファイル)やアノテーション情報(gffファイル)はサイズが大きいので大抵圧縮されてgzip形式になっています。

`gunzip`というコマンドで解凍が可能です。

In [ ]:
# 解凍
!gunzip Magnaporthe_oryzae.MG8.dna.toplevel.fa.gz
!gunzip Magnaporthe_oryzae.MG8.pep.all.fa.gz
!gunzip Magnaporthe_oryzae.MG8.60.gtf.gz

ファイルをメモ帳などで開いて中身を見ることも当然可能ですが、

`head`というコマンドを使うことで、ファイルの頭からn行目までを表示させることが出来ます。

```
head -n 表示する行数 表示させたいファイル名
```

まずは先ほどダウンロードしたファイルの最初の何行かを見てみましょう。

In [ ]:
# ゲノム配列の最初の5行を表示
!head -n 5 Magnaporthe_oryzae.MG8.dna.toplevel.fa

また、`mv 元のファイル名 新しいファイル名`というコマンドでファイル名を変更することも出来ます。

ファイル名が長くて以降の実習でややこしくなるので、ゲノム配列を`genome.fasta`、アミノ酸配列を`protein.fasta`、遺伝子アノテーション情報を`annotation.gtf`に変更しておきます。

In [ ]:
!mv Magnaporthe_oryzae.MG8.dna.toplevel.fa genome.fasta
!mv Magnaporthe_oryzae.MG8.pep.all.fa protein.fasta
!mv Magnaporthe_oryzae.MG8.60.gtf annotation.gtf

これで、イネいもち病菌のゲノム配列(genome.fasta)と全遺伝子のアミノ酸配列(protein.fasta)、遺伝子のアノテーション情報(annotation.gtf)が準備出来ました。

**※色々コマンドが出てきていますが、覚える必要は無く、そういうことが出来ると知っておけばOKです。**

### ソフトウェアのダウンロード・インストール

　次に、今回使用するソフトウェアのインストールをします。

本実習では、`seqkit`と`blast+`と`gffread`を使います。使い方はまた後程順に説明していきます。

今回はこちらでインストールするコマンドを準備しましたが、今後皆さんが自分のPC等の環境で行う際には、

マニュアルに書いてあるインストール方法を参考にしつつインストールしてみて下さい。

* `seqkit` ... https://bioinf.shenwei.me/seqkit/

* `blast+` ... https://www.ncbi.nlm.nih.gov/books/NBK279690/

* `gffread` ... https://ccb.jhu.edu/software/stringtie/gff.shtml#gffread

In [ ]:
%%bash
wget -q -O 'get_software2.sh' https://github.com/slt666666/NLR_biology_workshop_2024/raw/master/script/get_software2.sh
bash get_software2.sh
ls tools/

---
## 1. 必要な情報をFASTAから抜き出す(seqkit)

特定の遺伝子のCDS配列やアミノ酸配列が欲しい場合、

素直にやるなら、アミノ酸配列のFASTAファイルから目的の遺伝子IDを検索して配列をコピーする、という形で問題無く行えると思います。

しかし、複数の遺伝子の配列を取り出したい時や、特定の染色体の配列など長い配列をコピーする必要がある場合には少し手間になってきます。

このような場合、`seqkit`を使用することで簡単に目的の配列を取り出すことが出来ます。

<br>

**課題: MGG_01000T0という遺伝子のアミノ酸配列を抜き出す。**

まずは１つの遺伝子配列を取り出してみましょう。

```
# コマンド凡例
seqkit faidx FASTAファイル名 遺伝子名
```
このコマンドでFASTAファイルから遺伝子名の配列を取り出せます。


In [ ]:
!seqkit faidx protein.fasta MGG_01000T0

コマンドを動かした出力結果は、コマンドの最後に`「>」`を使ってファイルに書き出すことが出来ます。

```
# コマンド凡例
seqkit faidx FASTAファイル名 遺伝子名 > 任意のファイル名
```

In [ ]:
!seqkit faidx protein.fasta MGG_01000T0 > MGG_01000T0.fasta

また、通常だと長い配列は改行して出力されますが、`-w 0`というオプションを付けると、改行を無くして出力する事も出来ます。

```
# コマンド凡例
seqkit faidx -w 0 FASTAファイル名 遺伝子名 > 任意のファイル名
```

In [ ]:
!seqkit faidx -w 0 protein.fasta MGG_01000T0 > MGG_01000T0.fasta

**練習: 3という配列(3番染色体の配列)をgenome.fastaから取り出してファイルに書き出してみましょう**

XXXの部分を適宜書き換えてみてください。

In [ ]:
!seqkit faidx XXX XXX > chr3.fasta

**課題: 複数の遺伝子のアミノ酸配列をまとめて抜き出す**

次は、

* MGG_01000T0
* MGG_02000T0
* MGG_03000T0
* MGG_04000T0
* MGG_05000T0

の5つの配列を取り出してみます。

抜き出したい遺伝子のリストを書いたテキストファイルを作成して、下記のコマンドで抜き出すことが出来ます。

今回はあらかじめlist.txtというファイルを作成してあるので、これを使ってみてください。

```
# コマンド凡例
seqkit faidx -w 0 FASTAファイル名 -l 抜き出したい遺伝子リスト > 任意のファイル名
```

In [ ]:
!seqkit faidx -w 0 protein.fasta -l list.txt > Pickup.fasta

**練習: 1,2,3という配列(1~3番染色体の配列)をgenome.fastaから取り出してファイルに書き出してみましょう**

下記コマンドは、genome.fastaからlist.txtに書かれた配列を取り出すコマンドになります。

list.txtを適切なリストに書き換えて動かしてみてください。

In [ ]:
!seqkit faidx -w 0 genome.fasta -l list.txt > chr1_2_3.fasta

## 2. 遺伝子Aと相同性のある配列をFASTAから抜き出す(blast, seqkit)

Blastは[webサイト](https://blast.ncbi.nlm.nih.gov/Blast.cgi)で行うことが多いかもしれませんが、ツールとして活用することも可能です。(Local blastと言ったりします。)

[NCBIのblast](https://blast.ncbi.nlm.nih.gov/Blast.cgi)では、NCBIのデータベースに対して相同性の高い配列を探すことが出来ます。

一方、ツールとしてBlastを使うことで、任意の配列データの中から、相同性の高い配列を探すことが出来ます。

<img src="https://github.com/slt666666/NLR_biology_workshop_2024/blob/master/images/blast.png?raw=true" alt="artificial_selection" height="250px" align="middle">

今回は、Avr-Pikというエフェクターをコードする遺伝子(AB498879.1)の配列が、イネいもち菌のどの染色体にあるか探してみましょう。

AVR-Pikの配列は[ここ](https://www.ncbi.nlm.nih.gov/nuccore/AB498879)のものを利用しました([Yoshida et al., 2009](https://doi.org/10.1105/tpc.109.066324))。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2025/images/blast_sample.png?raw=true" alt="blast_sample" height="300px" align="middle">

<br>

Local blastを使う際には、まず検索対象の配列のデータベースを作成します。

```
# コマンド凡例
makeblastdb -in FASTAファイル名 -dbtype nucl -hash_index
```

今回は、ゲノム配列に対して検索をかけたいので、検索対象の配列は`genome.fasta`になります。




In [ ]:
!makeblastdb -in genome.fasta -dbtype nucl -hash_index

続いて、blastを実行します。塩基配列を検索するので`blastn`というコマンドを使用します。

検索したい配列(AVR-PikのCDS配列)は、`AVR_Pik_CDS.fasta`というファイルになります。

```
# コマンド凡例
blastn -db 先ほど作成したデータベース名(FASTAファイル名) -query 検索したい配列のFASTAファイル -out 任意のファイル名.txt

```

In [ ]:
!blastn -db genome.fasta -query AVR_Pik_CDS.fasta -out blastn_result.txt

デフォルトの設定だとblast結果が見にくいので、`-outfmt`というオプションを使うことで、結果のフォーマットを変えることも出来ます。

また、任意のe-valueを`-evalue`というオプションで設定して、相同性の高さの基準を決めることも可能です。

```
# コマンド凡例
blastn -db 先ほど作成したデータベース名(FASTAファイル名) -query 検索した配列のFASTAファイル -evalue 1e-30 -outfmt 6 -out 任意のファイル名.txt
```

In [ ]:
!blastn -db genome.fasta -query AVR_Pik_CDS.fasta -evalue 1e-30 -outfmt 6 -out blastn_result_tab.txt

各列の情報は検索した配列の名前、ヒットした配列の名前、一致した位置の割合、重なった長さ...等を表しています。

参考: https://www.metagenomics.wiki/tools/blast/blastn-output-format-6

**練習: アミノ酸配列の中からAVR-Pikの配列と相同性の高いものを探してみましょう**

まず、1行目でアミノ酸配列のデータベースを作ります。<small>※アミノ酸配列なので -dbtypeがnuclからproになります。</small>

先ほどと違って塩基配列ではなく、アミノ酸配列で検索するので`blastn`ではなく`blastp`になります。

また、検索したい配列(AVR-Pikのアミノ酸配列)は、`AVR_Pik_protein.fasta`というファイルになります。

XXXの部分を書き換えて動かしてみてください。


In [ ]:
!makeblastdb -in protein.fasta -dbtype prot -hash_index

!blastp -db XXX -query XXX -outfmt 6 -out blastp_result_tab.txt

結果を見ると、いくつか似ているアミノ酸をコードする遺伝子が見つかりました。

少しややこしいので詳細は省きますが、`awk`というコマンドを使うと、`2列目だけ抜き出す`みたいなことが出来ます。

```
# コマンド凡例
awk '{print $2}' tab区切りBlast結果.txt | uniq > 任意のファイル名.txt
```

このコマンドと先ほど扱った`seqkit`を組み合わせることで、

`blastでヒットした遺伝子の配列を取り出す。`といった作業をサッと行うことが出来ます。

In [ ]:
# blast結果から2列目のみ取り出す
!awk '{print $2}' blastp_result_tab.txt | sort | uniq > blast_gene_list.txt
# seqkitでblastの結果で出た名前の遺伝子の配列を取り出す
!seqkit faidx -w 0 protein.fasta -l blast_gene_list.txt > blast_gene_list.fasta

## 指定領域の配列をFASTAから抜き出す(seqkit)

`seqkit`を使うと、領域を指定して配列を取得する事も出来ます。

```
# コマンド凡例
seqkit faidx -w 0 FASTAファイル名 配列名:start-end > 任意のファイル名
```

先ほどAVR-Pikの配列でblastをした結果、2という配列(2番染色体)の 8229912-8229571がヒットしたかと思いますが、この領域の配列を取得してみましょう。

In [ ]:
!seqkit faidx -w 0 genome.fasta 2:8229912-8229571 > AVR_Pik_candidate.fasta

**練習: このAVR-Pikのプロモーター候補領域として、上記の領域の3000bp手前の領域の配列を取得してみましょう。**

XXXの値を書き換えて動かしてみてください。

In [ ]:
!seqkit faidx -w 0 genome.fasta 2:XXX-XXX > AVR_Pik_promoter_candidate.fasta

## 遺伝子のアノテーション情報(gtfファイル)から遺伝子の領域を抜き出す

`gtfファイル`や`gffファイル`には遺伝子のアノテーション情報が載っています。

In [ ]:
!head -n 20 annotation.gtf

これまでに習った様に、`seqkit`を使ってgtfファイルに記述してある遺伝子領域の配列を抜き出すことが出来ます。

In [ ]:
!seqkit faidx -w 0 genome.fasta 2:7482162-7483739 > MGG_10811_dna.fasta

しかし、ある遺伝子のアミノ酸配列を取り出すことを考えると
1. gtfファイルの情報を元に、複数のexonの領域を抜き出してくる
1. 抜き出したexonの領域をつなぎ合わせる
1. コドン表に従ってアミノ酸配列に変換する

といった形で手間がかかります。

そのため、gtfファイルの情報を配列を取り出す際には`gffread`というツールが良く使用されます。

`gffread`を使うことで、gtfやgffファイルに基づいてCDS配列やアミノ酸配列を簡単に取り出すことが出来ます。

例えば、指定領域内に含まれる遺伝子のCDS、アミノ酸配列を一気に抜き出すには以下の様なコマンドになります。

```
# コマンド凡例
$ gffread gtfファイル名 -g 対応するFASTAファイル名 -r 染色体名:start-end -x CDS配列の出力ファイル名 -y アミノ酸配列の出力ファイル名
```

In [ ]:
# 1番染色体の1~1mbpに位置する遺伝子のCDS配列とアミノ酸配列を取り出す
!gffread annotation.gtf -g genome.fasta -r 1:1-1000000 -x CDS_chr01_1mb.fasta -y pep_chr01_1mb.fasta

## seqkitで出来る他のこと

`seqkit`にはほかにも様々な機能があります。

FASTQファイルからFASTAファイルへの変換

In [ ]:
!seqkit fq2fa sample.fastq.gz > sample.fasta

FASTQファイルの統計値をみる

In [ ]:
!seqkit stats sample.fastq.gz

FASTAファイルから指定の長さ以上の配列だけを取り出す

In [ ]:
!seqkit seq -m 10000 sample.fasta > over10000.fasta

FASTAファイルの統計値をみる

In [ ]:
!seqkit stats -a over10000.fasta

配列をreverse complementaryする

In [ ]:
!seqkit seq -pr sample.fasta> reverse_complementary.fasta

---
## まとめ・補足

　ここでは`seqkit`と`blast+`と`gffread`を使って基本的なFASTAファイルの扱い方を試しました。

他にも配列データを扱うツールやコマンドは多数あり、より効率的にFASTAファイルやFASTQファイルを扱うことが可能になります。

大規模なゲノムデータを扱う場合には特に重要になってくるので、必要になれば勉強してみてください。